In [2]:
import numpy as np
import cv2
from datetime import datetime, timezone

import Segmentation.U_2_Net.u2net_test as u2net
import Segmentation.find_instances as find_instances
import Segmentation.top_finder as top_finder
import Robot_Control.block_stack as block_stack

# Our Code
import camera_control
import Robot_Control.block_stack as block_stack

from xarm.wrapper import XArmAPI

In [2]:
arm = XArmAPI('192.168.1.165')
arm.connect()

ROBOT_IP: 192.168.1.165, VERSION: v2.4.0, PROTOCOL: V1, DETAIL: 6,9,LI1003,DL1000,v2.4.0, TYPE1300: [0, 0]
change protocol identifier to 3


In [ ]:
# Model Parameters

max_iteration = 2000

# the movement area in robot coordinates and image coordinates.
robot_coords = np.float32([[120.6, -250.9], [126, 251.5], [349.4, -255.6], [352.2, 251.2]])
image_coords = np.float32([[0, 0], [1040, 0], [0, 520], [1040, 520]])

# find transformation matrices
M, top_points = camera_control.detect_aruco() #create transformation matrix and the top aruco coordinates
M_table_to_robot = cv2.getPerspectiveTransform(robot_coords, image_coords)

# current time for data collection
current_Time = str(datetime.now(timezone.utc))[:-13]
success, picked_centers =  [], []

In [ ]:
def create_placement(centers, chosen_block):
    
    randrz = np.random.randint(0, 90) # random rotation, symmetrical cubes
    
    while True:
        # pick a random placement for the block
        randx = np.random.randint(200, 300)
        randy = np.random.randint(-170, 170)

        flag_close = []
    
        for index in range(len(centers)):
            if index == chosen_block: continue
            else:
                center = centers[index]
                flag_close.append((np.abs(randx-center[0]) < 60) and (np.abs(randx-center[0]) > 30))\
                            and (np.abs(randy-center[1]) < 60) and (np.abs(randy-center[1]) > 30)
        if np.any(flag_close) or (len(centers) == 1):
            break
    return np.array([randx, randy, randrz])

In [ ]:
def main(i):
    while i<2000:
        # Save transformed image
        camera_control.transform_image(M, True)

        # Run the U_2_Net model
        u2net.main()

        # Separate Instances
        blocks = find_instances.run("camera_output.jpg", "./Segmentation/camera_output.png")

        corners,h_means = top_finder.run(blocks,M_table_to_robot)

        centers = []
        for set in range(len(corners)):
            center = np.average(corners[set], axis=0) - h_means[set]

            #Add random noise to the center point
            center = center + 10 * np.random.random()

            transformed_points = np.squeeze(cv2.perspectiveTransform(np.array(center).reshape(-1, 1, 2), M_table_to_robot))
            centers.append(transformed_points)


        # pick a random block from the list
        chosen_block = np.random.randint(len(centers))
        rand_block = np.random.randint(len(centers))
        
        # pick a random placement for the block that is far/close enough to other blocks
        rand_place = create_placement(centers, chosen_block)

        row_threshold = int(np.average([top_points[0][1], top_points[1][1]])) # any point above the row threshold is outside the workspace
        cols_threshold = np.array([int(top_points[0][0]), int(top_points[1][0])]) # any point outside the column thresholds is outside the workspace


        is_picked = block_stack.stack_func(centers[rand_block], rand_place[:2], rand_place[2], row_threshold, cols_threshold)
        if is_picked is not None: #didn't crash
            i+=1
            new_entry = []
            new_entry.append(int(is_picked)) #failed or successful pick
            new_entry.append(centers[rand_block][0])
            
            success.append(int(is_picked))s
            np.savetxt('Data/' + current_Time + '/label_'+current_Time+'.csv', np.array(success),  fmt='%i', delimiter=',')

            cv2.imwrite("Data/" + current_Time + "/run_number_" + str(i)+".jpg", blocks[rand_block])
            full_img = cv2.imread("camera_output.jpg")
            cv2.imwrite("Data/" + current_Time + "/run_number_fullimg" + str(i) + ".jpg", cv2.bitwise_and(full_img,full_img, mask=blocks[rand_block]))
            picked_centers.append(centers[rand_block])
            np.savetxt('Data/' + current_Time + '/picked_centers_' + current_Time + '.csv', np.array(picked_centers), fmt='%i', delimiter=',')

        block_stack.complete_clean()
        block_stack.set_initial_position()
        #return i

In [ ]:
# Define csv file saves data
file_name = 'Collected_Data/' + current_Time + '.csv'

with open(file_name, 'w', newline='') as f:
    pass

In [ ]:
i = 0
while (i<max_iteration): # Error Handler
    block_stack.complete_clean() # Clean any errors in the arm
    block_stack.set_initial_position() # Set the arm to the initial position at each iteration
    
    try: # trying to run iterations of block pickups
        i = main(i)
    except Exception as e:
        #TODO
        pass